In [90]:
import nltk
from textblob import TextBlob
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
import pymongo
import os
from pprint import pprint
import nytimes
import urllib, json, csv
from datetime import *
from dateutil.relativedelta import *
import calendar, time
from random import randint
from time import sleep

In [151]:
search_obj.article_search(filename="ph_2014.json",q="prospect heights real estate",begin_date='20140101',end_date='20141231')

In [76]:
search_obj = nytimes.get_article_search_obj ('7ff048af1779e0c12cf7038c3d9476a0:15:56968285')

In [202]:
def nb_name_formatter(path):
    '''creates a formatted list of borough names'''
    names = []
    with open(path, 'rU') as f:
        data = [row for row in csv.reader(f.read().splitlines())]
        for row in data:
            row = row
            names.extend(row)
        return names

def names_no_space(names):
        for i, n in enumerate(names):
            names[i-1] = n+'_'+code
        return names


f_path ='/Users/gfsoileau/Metis/Fletcher/queens_names.csv'

#bx_names = nb_name_formatter(f_path,'BX')
bx_no_space = names_no_space(bronx_names)
#qn_names = nb_name_formatter(f_path,'QN')
#qn_no_space = names_no_space(qn_names)
#mn_names = nb_name_formatter(f_path, 'NY')
#mn_no_space = names_no_space(mn_names)


NameError: global name 'code' is not defined

In [196]:
print queens_names

['Alley Park', 'Astoria', 'Auburndale', 'Bayside', 'Bay Terrace', 'Bellerose', 'Bowne Park', 'Briarwood', 'Brookville', 'Cambria Heights', 'Clearview', 'College Point', 'Corona', 'Ditmars-Steinway', 'Douglaston', 'East Elmhurst', 'Elmhurst', 'Floral Park', 'Flushing', 'East Flushing', 'Murray Hill', 'Forest Hills', 'Fresh Meadows', 'Glen Oaks Village', 'Glendale', 'Hillcrest', 'Hollis', 'Howard Beach', 'Hamilton Beach', 'Howard Park', 'Lindenwood', 'Old Howard Beach', 'Ramblersville', 'Rockwood Park', 'Jackson Heights', 'Jamaica', 'Jamaica Estates', 'Jamaica Hills', 'Kew Gardens', 'Kew Gardens Hills', 'Laurelton', 'Little Neck', 'Long Island City', 'Hunters Point', 'Maspeth', 'Middle Village', 'New Hyde Park', 'North Corona', 'Oakland Gardens', 'Ozone Park', 'Pomonok', 'Queens Village', 'Queensboro Hill', 'Rego Park', 'Richmond Hill', 'Ridgewood', 'Rockaway All', 'Arverne', 'Bayswater', 'Belle Harbor', 'Breezy Point', 'Broad Channel', 'Edgemere', 'Far Rockaway', 'Hammels', 'Neponsit', 

In [169]:
years = [2015,2014,2013,2012,2011,2010,2009,2008,2007]

BK_names = ['Bay Ridge','Bedford-Stuyvesant','Bensonhurst','Bergen Beach','Boerum Hill','Borough Park','Brighton Beach',
           'Brooklyn Heights','Brooklyn Navy Yard','Brownsville','Bushwick','Canarsie','Carroll Gardens',
           'Clinton Hill','Cobble Hill','Coney Island','Crown Heights',
           'Cypress Hills','Ditmas Park','Downtown Brooklyn','DUMBO','Dyker Heights','East New York',
           'East Williamsburg','Farragut','Flatbush','Flatlands','Fort Greene','Fort Hamilton','Gerritsen Beach',
           'Gowanus','Gravesend','Greenpoint','Greenwood Heights','Kensington',
           'Manhattan Beach','Marine Park','Midwood','Mill Basin','New Lots','New Utrecht',
           'Ocean Parkway','Old Mill Basin','Park Slope','Prospect Heights',
           'Prospect Lefferts Gardens','Prospect Park South','Red Hook','Seagate','Sheepshead Bay',
           'Stuyvesant Heights','Sunset Park','Vinegar Hill','White Sands','Williamsburg','Windsor Terrace']


In [198]:
def nyt_api(names, years):
    '''calls nytimes api with a formatted query string and saves json file '''
    for n in names:
        for y in years:
            search_obj.article_search(filename="'"+n+str(y)+".json" "'",q="'"+n+" real estate" "'",begin_date=str(y)+"0101",end_date=str(y)+"1231")  
            
nyt_api(queens_names,years)

ValueError: No JSON object could be decoded

In [192]:
def format_nyt_dict(path, borough):
    '''format the json files return from the API and insert into mongodb nytimesdb, neighborhood_data.collection.
        Takes the path to directory where files are stored and the borough name'''
    files = os.listdir(path)
    for f in files:
        year = f[-10:-6]
        name_hood = f[1:-10]
        print year, name_hood
        f_path ='/Users/gfsoileau/Metis/Fletcher/times_data/'+f
        with open(f_path) as json_file:
            print '*' *20
            data = json_file.read().replace('\n', '')
            times_dict = json.loads(data)
            count = 0
    #         for k, v in times_dict['response']['meta'].items():
    #             dic[k]=v 
            for i in times_dict['response']['docs']:
                dic = {}
                if i['news_desk'] == 'RealEstate' or i['news_desk'] == 'Metro'and i["section_name"] != "Great Homes & Destinations":
                    count +=1
                    dic['id'] = i['_id']
                    if i['abstract'] is not None:
                        dic['abstract'] = i['abstract'].encode("utf8")
                    dic['headline'] = i['headline']['main'].encode("utf8")
                    dic['desk'] = i['news_desk']
                    dic['date'] = i['pub_date'][0:10] # cutting time of day.
                    dic['section'] = i['section_name']
                    if i['snippet'] is not None:
                        dic['snippet'] = i['snippet'].encode("utf8")
                    dic['source'] = i['source']
                    dic['url'] = i['web_url']
                    dic['word_count'] = i['word_count']
                    dic['neighborhood'] = name_hood
                    dic['year'] = year
                    dic['borough'] = borough
                    dic['count'] = count 
                else:
                    continue
                print dic
                nb.insert(dic)
                
path = '/Users/gfsoileau/Metis/Fletcher/times_data'
format_nyt_dict(path,'Bronx')

2007 Windsor Terrace
2008 Windsor Terrace
2009 Windsor Terrace
.DS 
 
 


In [165]:

# meta_dict = {} 
# for f in files:
#     year = f[-10:-6]
#     name_hood = f[1:-10]
#     f_path ='/Users/gfsoileau/Metis/Fletcher/times_data/'+f
#     with open(f_path) as json_file:
#         data = json_file.read().replace('\n', '')
#         times_dict = json.loads(data)
#         for k, v in times_dict['response']['meta'].items():
#             meta_dict[k]=v
            
# print meta_dict

{u'hits': 16, u'offset': 0, u'time': 75}


In [ ]:
try:
    conn=pymongo.MongoClient()
    print "Connected successfully!!!"
except pymongo.errors.ConnectionFailure, e:
    print "Could not connect to MongoDB: %s" % e 

In [172]:
#create ny_times db and alias it to db
db = conn.ny_times
# conn.database_names()

In [171]:
db.collection_names()

[u'neighborhood_data', u'street_easy', u'system.indexes']

In [162]:
#drop neighborhood_data collection
conn.ny_times.drop_collection('neighborhood_data')

In [20]:
#create neighborhood_data collection
nb = db.neighborhood_data

In [55]:
es = db.street_easy

In [17]:
test = db.test_collection

In [177]:
#es.find_one()
#list(es.find())[50:110]
#conn.ny_times.drop_collection('street_easy')

In [178]:
#list(nb.find())[1000:1010]

In [251]:
cursor = nb.find({"neighborhood":"Park Slope", 'year':'2010'})

In [34]:
hood_names = ['Boerum_Hill','Borough_Park','Brighton_Beach',
           'Brooklyn_Heights','Brooklyn_Navy_Yard','Brownsville','Bushwick','Canarsie','Carroll_Gardens',
           'Clinton_Hill','Cobble_Hill','Coney_Island','Crown_Heights',
           'Cypress_Hills','Ditmas_Park','Downtown_Brooklyn','DUMBO','Dyker_Heights','East_New_York',
           'East_Williamsburg','Farragut','Flatbush','Flatlands','Fort_Greene','Fort_Hamilton','Gerritsen_Beach',
           'Gowanus','Gravesend','Greenpoint','Greenwood_Heights','Kensington',
           'Manhattan_Beach','Marine_Park','Midwood','Mill_Basin','New_Lots','New_Utrecht',
           'Ocean_Parkway','Old_Mill_Basin','Park_Slope','Prospect_Heights',
           'Prospect_Lefferts_Gardens','Prospect_Park_South','Red_Hook','Seagate','Sheepshead_Bay',
           'Stuyvesant_Heights','Sunset_Park','Vinegar_Hill','White_Sands','Williamsburg','Windsor_Terrace']
#'Bay_Ridge','Bedford_Stuyvesant','Bensonhurst','Bergen_Beach'(notfinished)

In [36]:
def create_hits_list(hood_list, year_sample):
    '''creates a list of dictionaries of total ny_times hits per year for a neighborhood'''
    num_articles = []
    for h in hood_list:
        for y in year_sample:
            num_articles_year = 0
            cursor = nb.find({'neighborhood':h, 'year':y}, {'count':1,'_id':False})
            for i in cursor:
                num_articles_year = i
            num_articles.append(num_articles_year)
            
    return num_articles

print create_hits_list(hood_sample,year_sample)


[{u'count': 9}, {u'count': 9}, {u'count': 4}, {u'count': 3}, {u'count': 5}]


In [37]:
def hits_list(hood_list):
    '''transform num_articles_all into a list of values from dictionary'''
    for idx, val in enumerate(hood_list):
         for k,v in val.iteritems():
            hood_list[idx] = v
    return hood_list
       
print hits_list(num_articles_all)    

NameError: name 'num_articles_all' is not defined

In [75]:
api_keys = ['8670b57035a194e0772b3ff0b47669567ed895d0','6bfd4448e5a38a9d7dbd2555e7663d7bfde315b1'
            '2a0c1d73b28f373a660de59f408471967c9bfd91','ff70b20c1c4e0a4b0fcbd45a4a97dcd0fceebdc3',
            'dc87c720b13afec5f010dffe7d3ee6aac0539ef4','bd9f608751bdad74e946d4dba73e309d4b05894b',
            '14d0372671c71f144d24699998a0670482238f5c','71addf314166bb097a6bb4d81c75680c6048da15',
            '2d2fb018b6a1d485efcc0be4f2456edec6d0d9ec','107e6df6dd9d259cb7edd2408246920817f643ca']

def format_hoods(hood_list):
    '''remove all spaces in BK neighborhood names, not needed for other neighborhoods'''
    formatted_hoods = []
    for h in hood_list:
        formatted_h = h.replace(' ','')
        formatted_hoods.append(formatted_h)
    return formatted_hoods 
#formatted_hoods = format_hoods(q_terms)


 data = json_file.read().replace('\n', '')
            times_dict = json.loads(data)
            for i in times_dict['response']['docs']:
                dic = {}
                if i['news_desk'] == 'RealEstate' or i['news_desk'] == 'Metro'and i["section_name"] != "Great Homes & Destinations":
                    count +=1
                    dic['id'] = i['_id']
                    if i['abstract'] is not None:
                        dic['abstract'] = i['abstract'].encode("utf8")
                    dic['headline'] = i['headline']['mai



def get_street_easy(neighborhoods,keys, borough):
            count =1
            api_counter = 0
            st_easy_dict = {}
            today = date.today()
            today = today+relativedelta(months = -3,days = -2)
            year = dt[0:4]
            month = dt[4:6]
                                                    
#             for n in neighborhoods:
#                 for i in range(1,97):
#                     if count <= 97 and api_counter <= 9: 
#                         start_month = today
#                         end_month = today+relativedelta(months=-1)
#                         end_dt = str(start_month).replace('-','')
#                         start_dt = str(end_month).replace('-','')
#                         today = end_month
#                         #print count, 'http://streeteasy.com=area:'+h+'-brooklyn|status:sold%7Clisted_between:'+start_dt+'-'+end_dt+'key='+keys[api_counter]
#                         count += 1
#                         url = 'http://streeteasy.com/nyc/api/sales/data?criteria=area:'+n+'-brooklyn|status:sold%7Clisted_between:'+start_dt+'-'+end_dt+'&key='+keys[api_counter]+'&format=json'
#                         try:
#                             response = urllib.urlopen(url);
#                             data = json.loads(response.read())
#                             print count, url, data
#                             #es.insert(data)
#                             data['borough'] = borough
#                             data['year'] = start_dt
                                                    
                                                    
#                             for i in data['response']['docs']:
#                                 dic = {}
#                                 if i['news_desk'] == 'RealEstate' or i['news_desk'] == 'Metro'and i["section_name"] != "Great Homes & Destinations":
#                                     count +=1
#                                     dic['id'] = i['_id']
#                                     if i['abstract'] is not None:
                                                    
                                                    
                                                    
#                             time.sleep(5)
#                         except:
#                             print "Could not get json: %s" % e, url
#                     else:
#                         if api_counter < 9:
#                             api_counter += 1
#                             count = 0
#                             print '*************************change key*************************************'
#                         else:
#                             print '**************************sleep and restart*****************************'
#                             time.sleep(1800) 
#                             api_counter = 0
#                             count = 0
                            
#get_street_easy(formatted_hoods,api_keys,'Brooklyn')    

2 http://streeteasy.com/nyc/api/sales/data?criteria=area:Canarsie-brooklyn|status:sold%7Clisted_between:20150202-20150302&key=8670b57035a194e0772b3ff0b47669567ed895d0&format=json {u'stddev_increase': 0.0, u'average_sqft': 1275, u'updated_at': u'2015-06-04T11:27:40-04:00', u'average_wom': 4.0, u'median_price': 409000, u'median_ppsf': 316, u'search_url': u'http://streeteasy.com/for-sale/canarsie/status:sold%7Clisted_between:20150202-20150302', u'average_ppsf': 313, u'stddev_decrease': 0.0, u'average_increase': 0.0, u'decrease_count': 1, u'criteria': u'status:sold|area:Canarsie-brooklyn|listed_between:20150202-20150302', u'listing_count': 3, u'wom_count': 2, u'increase_count': 0, u'stddev_wom': 2.8284271247461903, u'stddev_sqft': 247.48737341529164, u'sqft_count': 2, u'criteria_description': u'Sale sold listings in Canarsie  listed between 2015-02-02 and 2015-03-02  ', u'median_wom': 4.0, u'median_sqft': 1275, u'average_decrease': 105.0, u'percentile_10_price': 375000, u'average_price': 4

IndexError: list index out of range

In [208]:
dt = "20150102"
dt2 = dt[0:4]
month = dt[4:6]
print dt2, month

2015 01
